In [4]:
import numpy as np
import scipy as sp
import pandas as pd
import sklearn
import numpy.linalg as lin
import sklearn.cross_validation
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.datasets import load_svmlight_file
from sklearn.preprocessing import Normalizer
import fileinput
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Binarizer
from sklearn.cluster import KMeans

import xgboost as xgb
import pickle

from sklearn.linear_model import LogisticRegression

In [5]:
dr='auto/'
col = 'auto_researcher'
#dr = 'elec/'
#col = 'consumer_electronics'
#dr = 'fin/'
#col = 'financial_service'

In [12]:
user = pd.read_csv('u360_demodata.tsv', sep='\t', header=None, usecols=[0,1,3,6], names=['UserID', 'Country', 'Genre','Age'])

l=[]
f = fileinput.input([dr + 'train_category_libsvm.dat'])
for line in f:
    l.append(int(line.split(' ', 1)[0]))
f.close()
df_tr = pd.merge(pd.DataFrame({'UserID':l}), user, how="left", on="UserID")
df_tr.fillna('nan', inplace=True)

Xtr, Ytr = load_svmlight_file(dr + "train_category_libsvm_2.dat")

f = open("DisplayAds_params.pk", 'wb')
params = {}
params['ncol']=Xtr.shape[1]
pickle.dump(params,f)
f.close()

cat_cols = ['Country','Genre','Age']
for c in cat_cols:
    l = np.unique(list(df_tr[c].values))
    f = open("DisplayAds_" + c + ".pk", 'wb')
    dt = {k:v for (k, v) in zip(l, range(1,len(l)+1)) if v!='nan'}
    dt['nan'] = 0
    pickle.dump(dt,f)
    f.close()
    df_tr[c] = df_tr[c].apply(lambda x:dt[x])

enc = OneHotEncoder()
enc.fit(df_tr[cat_cols])
f = open("DisplayAds_Encoder.pk", 'wb')
pickle.dump(enc,f)              
f.close()
                  
Xtr = sp.sparse.hstack([enc.transform(df_tr[cat_cols].values), Xtr], format='csr')

f =  open("DisplayAds_Normalizer.pk", 'wb')
nr = Normalizer()
_Xtr = nr.fit_transform(Xtr)
pickle.dump(nr, f)              
f.close()
                  
f =  open("DisplayAds_Clf.pk", 'wb')
clf = LogisticRegression(C=0.2,class_weight='auto', random_state=0)
clf.fit(_Xtr, Ytr)
pickle.dump(clf, f)              

In [13]:
user = pd.read_csv('u360_demodata.tsv', sep='\t', header=None, usecols=[0,1,3,6], names=['UserID', 'Country', 'Genre','Age'])

l=[]
f = fileinput.input([dr + 'test_category_libsvm.dat'])
for line in f:
    l.append(int(line.split(' ', 1)[0]))
f.close()
userids=np.array(l)
df_te = pd.merge(pd.DataFrame({'UserID':l}), user, how="left", on="UserID")
df_te.fillna('nan', inplace=True)

Xte, Yte = load_svmlight_file(dr + "test_category_libsvm_2.dat")

f = open("DisplayAds_params.pk", 'rb')
params = pickle.load(f)
Xte = Xte[:,:params['ncol']]
f.close()

cat_cols = ['Country','Genre','Age']
for c in cat_cols:
    f = open("DisplayAds_" + c + ".pk", 'rb')
    dt = pickle.load(f)
    df_te[c] = df_te[c].apply(lambda x:dt[x] if x in dt.keys() else 0)
    f.close()
    
f = open("DisplayAds_Encoder.pk", 'rb')
enc = pickle.load(f)
Xte = sp.sparse.hstack([enc.transform(df_te[cat_cols].values), Xte], format='csr')
f.close()

f =  open("DisplayAds_Normalizer.pk", 'rb')
nr = pickle.load(f)
_Xte = nr.transform(Xte)
f.close()

f =  open("DisplayAds_Clf.pk", 'rb')
clf = pickle.load(f)
y_p = [x[1] for x in clf.predict_proba(_Xte)]
f.close()

In [14]:
testLabel = pd.read_csv(dr + "testLabel.dat", sep='\t')
testLabel.sort('UserID',inplace=True)
print roc_auc_score(testLabel.Label.values, y_p)

0.721459883879


In [77]:
#0.721457931885
#0.684729975559
#0.589011822398

